In [1]:
import requests as r
import random

In [17]:
def gen_event():
    event = {'lat': 70 + random.random()*5,
             'lon': 50 + random.random()*8,
             'ts': 1546293673 + random.randint(-1000, 1000),
             'event': 0}
    return event

In [18]:
url = 'http://0.0.0.0:5000/'

In [32]:
method = 'processEvents'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

[{'cid': -1,
  'event': 0,
  'lat': 74.44100668746476,
  'lon': 55.93196740050959,
  'ts': 1546294508},
 {'cid': -1,
  'event': 0,
  'lat': 73.64468757525998,
  'lon': 56.56591075485368,
  'ts': 1546294262},
 {'cid': -1,
  'event': 0,
  'lat': 72.48048365358439,
  'lon': 55.78518169580002,
  'ts': 1546293812}]

In [33]:
method = 'isClusterEmergency'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

{'prob': 0}

In [34]:
method = 'getClusterEmergencyReasons'

resp = r.get(url+method, json=[gen_event() for _ in range(3)])
eval(resp.text)

{'reasons': {'unusual_high_activity': {'importance': 0, 'perc': 3.0},
  'unusual_pressure': {'importance': 0, 'perc': 1},
  'unusual_temperature': {'importance': 0, 'perc': 1},
  'unusual_visibility': {'importance': 0, 'perc': 1},
  'unusual_wetness': {'importance': 0, 'perc': 1}}}

# Develop check emergency

In [7]:
import pandas as pd
import time
to_timestamp = lambda strdate: int(time.mktime(time.strptime(str(strdate), '%Y-%m-%d %H:%M:%S')))
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from models.cdbscan import Event, ContinuosCluster
from models.models import ClusteringModel

In [8]:
df = pd.read_excel('../data/raw/new_Dataset3_Dtp.xlsx')
df['timestamp'] = list(map(to_timestamp, df['Время регистрации']))
df['lat'] = df['Широта']
df['lon'] = df['Долгота']
df = df['timestamp lat lon T P U VV'.split()]
df = df[df.isna().sum(axis=1) == 0]
df.head()

,timestamp,lat,lon,T,P,U,VV
0,1546293673,59.937400,30.282000,-1.7,761.6,79.0,10.0
1,1546293709,59.936924,30.282853,-1.7,761.6,79.0,10.0
2,1546293713,59.946515,30.271703,-1.7,761.6,79.0,10.0
3,1546293725,59.945196,30.273266,-1.7,761.6,79.0,10.0
4,1546293760,59.945800,30.271000,-1.7,761.6,79.0,10.0


In [10]:
(df.mean()-1) / (df.mean()+1)

timestamp    1.000000
lat          0.964387
lon          0.930828
T            0.811452
P            0.997367
U            0.970900
VV           0.805502
dtype: float64

In [ ]:
def build_events(events_data):
    events = []
    for (ts, lat, lon) in events_data:
        events.append(Event(lat, lon, ts))
    return events

In [ ]:
last_ts = 1576200000
subdf = df[df.timestamp < last_ts]
subdf.shape, df.shape

In [ ]:
interval = 60*60*4 # by two hours

In [ ]:
start_ts = int(subdf.loc[0].timestamp.item())
stop_ts = int(subdf.loc[len(subdf)-1].timestamp.item())
model = ClusteringModel(0.01, 4, 60*10)

In [ ]:
# explore avg cluster info
avg_clusters_timestamps = []
avg_clusters = []

for start in tqdm(range(start_ts, stop_ts, interval//2)):
    events = subdf[(subdf.timestamp > start) & (subdf.timestamp < start+interval)]
    events = build_events(events.to_records(index=False))
    
    if len(events) < 2:
        continue
    
    labels = model.fit(events)
    clusters = [ContinuosCluster([]) for _ in range(max(0, max(labels)+1))]
    for i, event in enumerate(events):
        if labels[i] != -1:
            clusters[labels[i]].add(event)
    
    avg_clusters.append(clusters)
    avg_clusters_timestamps.append(start_ts + interval//2)

In [ ]:
# clusters count
_ = plt.figure(figsize=(20, 4))
_ = plt.plot(list(map(len, avg_clusters)))

In [ ]:
# avg points in clusters
_ = plt.figure(figsize=(20, 4))
def get_avg_points_n(avg_clusters):
    mean = lambda arr: sum(arr)/max(1, len(arr))
    avg_points_n = []
    for clusters in avg_clusters:
        avg_p_n = mean(list(map(lambda cl: len(cl.events), clusters)))
        avg_points_n.append(avg_p_n)
    return avg_points_n
        
_ = plt.plot(get_avg_points_n(avg_clusters))